In [2]:
from dotenv import load_dotenv, find_dotenv
import os
# import pprint
# import missingno as msno
from pymongo import MongoClient
import pandas as pd 
import numpy as np
import regex as re


In [22]:
# au_app_df = pd.read_csv('../data/power_bi/individuals/au_app_df.csv')
# au_app_num_df = pd.read_csv('../data/power_bi/individuals/au_app_num_df_more_than_10.csv')
# au_rank_df = pd.read_csv('../data/power_bi/individuals/au_rank_df.csv')

# kr_app_df = pd.read_csv('../data/power_bi/individuals/ko_app_df_en.csv')
# kr_app_num_df = pd.read_csv('../data/power_bi/individuals/ko_app_num_df_more_than_10.csv')
# kr_rank_df = pd.read_csv('../data/power_bi/individuals/ko_rank_df.csv')

In [23]:
# dec_au_app_df = pd.read_csv('../data/power_bi/individuals/december/12_au_app_df.csv')
# dec_au_app_num_df = pd.read_csv('../data/power_bi/individuals/december/12_au_app_num_df_10.csv')
# dec_au_rank_df = pd.read_csv('../data/power_bi/individuals/december/12_au_rank_df.csv')

# dec_kr_app_df = pd.read_csv('../data/power_bi/individuals/december/12_kr_app_df.csv')
# dec_kr_app_num_df = pd.read_csv('../data/power_bi/individuals/december/12_kr_app_num_df_10.csv')
# dec_kr_rank_df = pd.read_csv('../data/power_bi/individuals/december/12_kr_rank_df.csv')

In [ ]:
# au_app_df = dec_au_app_df.copy(deep=True)
# au_app_num_df = dec_au_app_num_df.copy(deep=True)
# au_rank_df = dec_au_rank_df.copy(deep=True)

# kr_app_df = dec_kr_app_df.copy(deep=True)
# kr_app_num_df = dec_kr_app_num_df.copy(deep=True)
# kr_rank_df = dec_kr_rank_df.copy(deep=True)

In [3]:
au_app_df = pd.read_csv('../data/power_bi/individuals/december/12_au_app_df.csv')
au_app_num_df = pd.read_csv('../data/power_bi/individuals/december/12_au_app_num_df_10.csv')
au_rank_df = pd.read_csv('../data/power_bi/individuals/december/12_au_rank_df.csv')

kr_app_df = pd.read_csv('../data/power_bi/individuals/december/12_kr_app_df.csv')
kr_app_num_df = pd.read_csv('../data/power_bi/individuals/december/12_kr_app_num_df_10.csv')
kr_rank_df = pd.read_csv('../data/power_bi/individuals/december/12_kr_rank_df.csv')

In [ ]:
# kr_app_df['released'][kr_app_df['app_id'] == 'com.ntori.Jicheon'] = '2023-12-12'
# kr_app_df['released'][kr_app_df['app_id'] == 'com.intsig.BizCardReader'] = '2023-12-17'
# kr_app_df['released'][kr_app_df['app_id'] == 'com.speedsoftware.rootexplorer'] = '2023-10-13'
# kr_app_df['released'][kr_app_df['app_id'] == 'com.facebook.katana'] = '2007-01-10'
# kr_app_df['released'][kr_app_df['app_id'] == 'com.google.android.apps.translate'] = '2010-01-01'

# kr_app_df.to_csv('12_kr_app_df.csv', index = False)

In [4]:
au_app_df['country'] = 'Australia'
au_app_num_df['country'] = 'Australia'
au_rank_df['country'] = 'Australia'
kr_app_df['country'] = 'Korea'
kr_app_num_df['country'] = 'Korea'
kr_rank_df['country'] = 'Korea'

In [20]:
merged_app_df = pd.concat([au_app_df, kr_app_df])
merged_app_num_df = pd.concat([au_app_num_df, kr_app_num_df])
merged_rank_df = pd.concat([au_rank_df, kr_rank_df])

In [21]:
merged_app_df['ko_genre'] = 0
merged_app_df['ko_contentRating'] = 0
merged_app_df['ko_min_price'] = 0
merged_app_df['ko_max_price'] = 0

# Mark duplciated app rows
merged_app_df['app_dup'] = 0
merged_app_df['app_dup'][merged_app_df.duplicated(subset='app_id', keep = False)] = 1 
merged_app_df['app_dup'].value_counts()

C:\Users\GIS\AppData\Local\Temp\ipykernel_18900\3776679345.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_app_df['app_dup'][merged_app_df.duplicated(subset='app_id', keep = False)] = 1


0    2598
1    1204
Name: app_dup, dtype: int64

In [23]:
# For the duplicated values, Australia comes first and South Korea comes next 
merged_app_df = merged_app_df.sort_values(['app_id', 'country']).reset_index(drop = True)

# Check if South Korea comes right after Australia. If there are only -1s, then it is true
au_index = merged_app_df[(merged_app_df.duplicated(subset='app_id', keep = False)) & (merged_app_df['country'] == 'Australia')].index
ko_index = merged_app_df[(merged_app_df.duplicated(subset='app_id', keep = False)) & (merged_app_df['country'] == 'Korea')].index
idx = au_index - ko_index
set(idx)

{-1}

In [34]:
# Put duplicated app's info that country is South Korea to duplicated apps' rows that country is Australia

#                   app_id                          title	date_collected	free  currency	developer	        email	            address	                                        genre   contentRating	released   min_price max_price	dev_country	   country	app_dup	ko_genre  ko_contentRating	ko_min_price	ko_max_price
# 2	age.of.civilizations2.jakowski.lukasz	Age of History II	2023-08-15	False	USD	Łukasz Jakowski	jakowskidev@gmail.com	twitter: @jakowskidev łukasz jakowski games pi...	Strategy	Everyone	2018-12-04	0.000	0.00	          uk	  Australia	    1	    0	        0	            0.000	    0.00
# 3	age.of.civilizations2.jakowski.lukasz	Age of History II	2023-08-10	False	KRW	Łukasz Jakowski	jakowskidev@gmail.com	twitter: @jakowskidev łukasz jakowski games pi...	Strategy	3 years+	2018-12-04	0.000	0.00	          uk	  South Korea	1	    0	        0	            0.000	    0.00
# It turns into:
#                   app_id                          title	date_collected	free  currency	developer	        email	            address	                                        genre   contentRating	released	min_price	max_price	dev_country	country	    app_dup	    ko_genre	ko_contentRating	ko_min_price	ko_max_price
# 2	age.of.civilizations2.jakowski.lukasz	Age of History II	2023-08-15	False	USD	Łukasz Jakowski	jakowskidev@gmail.com	twitter: @jakowskidev łukasz jakowski games pi...	Strategy	Everyone	2018-12-04	  0.000	        0.00	    uk	    Australia	    1	    Strategy	  3 years+	            0.000	        0.00

for i in au_index:
    merged_app_df.loc[i, 'ko_genre'] = merged_app_df.loc[i+1, 'en_genre']
    merged_app_df.loc[i, 'ko_contentRating'] = merged_app_df.loc[i+1, 'en_content_rating']
    merged_app_df.loc[i, 'ko_min_price'] = merged_app_df.loc[i+1, 'min_price']
    merged_app_df.loc[i, 'ko_max_price'] = merged_app_df.loc[i+1, 'max_price']


In [24]:
print(len(merged_app_df))

3802


In [35]:
merged_app_df[merged_app_df['title'] == 'ChatGPT']

,app_id,title,date_collected,free,currency,developer,email,address,genre,content_rating,...,max_price,dev_country,country,en_genre,en_content_rating,ko_genre,ko_contentRating,ko_min_price,ko_max_price,app_dup
2022,com.openai.chatgpt,ChatGPT,2023-08-10,True,USD,OpenAI,support@openai.com,none,Productivity,Teen,...,19.99,0,Australia,NaN,NaN,Productivity,12 years+,21.46,21.46,1
2023,com.openai.chatgpt,ChatGPT,2023-08-10,True,KRW,OpenAI,support@openai.com,none,생산성,12세 이상,...,21.46,0,Korea,Productivity,12 years+,0,0,0.00,0.00,1


In [30]:
print(len(merged_app_df[merged_app_df.duplicated(subset='app_id', keep = False)]))

1204


In [38]:
3802 - (1204/2)

3200.0

In [39]:
merged_app_df.drop(index = ko_index, inplace = True)
merged_app_df = merged_app_df.reset_index(drop = True)
print(len(merged_app_df))
merged_app_df.head()

3200


,app_id,title,date_collected,free,currency,developer,email,address,genre,content_rating,...,max_price,dev_country,country,en_genre,en_content_rating,ko_genre,ko_contentRating,ko_min_price,ko_max_price,app_dup
0,OCTech.Mobile.Applications.TouchScan,OBD Fusion (Car Diagnostics),2023-09-08,False,USD,"OCTech, LLC",service@obdsoftware.net,"4098 gibsonia road #215 gibsonia, pa 15044",Communication,Everyone,...,74.99,USA,Australia,NaN,NaN,0,0,0.000,0.00,0
1,adaptive.toons.app,Adaptive Toons Icon Pack,2023-10-10,False,USD,SKwallpapers,msk8898255565@gmail.com,"mumbai, maharashtra.",Personalization,Everyone,...,0.00,0,Australia,NaN,NaN,0,0,0.000,0.00,0
2,age.of.civilizations2.jakowski.lukasz,Age of History II,2023-08-15,False,USD,Łukasz Jakowski,jakowskidev@gmail.com,twitter: @jakowskidev łukasz jakowski games pi...,Strategy,Everyone,...,0.00,UK,Australia,NaN,NaN,Strategy,3 years+,0.000,0.00,1
3,ai.art.generator.paint.draw.photo,"AI Art Generator: Photo, Draw",2023-08-11,True,USD,Smart AI DEV,summerdaysc@outlook.com,none,Art & Design,Everyone,...,39.99,0,Australia,NaN,NaN,Art/Design,3 years+,3.182,43.66,1
4,ai.chat.gpt.bot,ChatOn - 지능적인 AI 챗봇 한글버전,2023-12-13,True,KRW,AIBY Inc.,support@aiby.com,"7901 4th street n ste 4913 st petersburg, fl 3...",생산성,3세 이상,...,62.90,USA,Korea,Productivity,3 years+,0,0,0.000,0.00,0


In [41]:
merged_app_num_df.columns

Index(['app_id', 'date_collected', 'real_installs', 'score', 'ratings',
       'reviews', 'score_1', 'score_2', 'score_3', 'score_4', 'score_5',
       'price', 'install_diff', 'score_diff', 'rating_diff', 'review_diff',
       'score_1_diff', 'score_2_diff', 'score_3_diff', 'score_4_diff',
       'score_5_diff', 'country'],
      dtype='object')

In [57]:
np.nan

nan

In [62]:
merged_app_df['date_collected'] = pd.to_datetime(merged_app_df['date_collected'])
merged_app_df['released'] = pd.to_datetime(merged_app_df['released'].apply(lambda x: np.nan if x == '-1' else x))
au_app_df['released'] = pd.to_datetime(au_app_df['released'].apply(lambda x: np.nan if x == '-1' else x))
kr_app_df['released'] = pd.to_datetime(kr_app_df['released'].apply(lambda x: np.nan if x == '-1' else x))


In [63]:
merged_app_df.to_csv('merged_app_df.csv', index = False)
merged_app_num_df.to_csv('merged_app_num_df.csv', index = False)
merged_rank_df.to_csv('merged_rank_df.csv', index = False)
au_app_df.to_csv('12_au_app_df.csv', index = False)
kr_app_df.to_csv('12_kr_app_df.csv', index = False)
